In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
import glob
import random
import collections
from glob import glob
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import shutil
import keras
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
%matplotlib inline

from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras import layers
from keras import models
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

/Users/sadianasrintisha/anaconda3/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [68]:
!pip3 install https://download.pytorch.org/whl/cpu/torch-1.0.1-cp36-cp36m-win_amd64.whl
!pip3 install torchvision

ERROR: torch-1.0.1-cp36-cp36m-win_amd64.whl is not a supported wheel on this platform.
  Using cached torchvision-0.15.2-cp310-cp310-macosx_11_0_arm64.whl (1.4 MB)
  Using cached torch-2.0.1-cp310-none-macosx_11_0_arm64.whl (55.8 MB)


In [3]:
def get_f1(y_true, y_pred): #taken from old keras source code
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * (y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1-y_pred), 0, 1)))
    

    f1_val = tp / ( tp + ( (1/2) * (fp+fn) ) + K.epsilon())
    return f1_val

# Data Upload

In [4]:
trn1='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/noninvasive/*/'
trn2='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracods1/*/'
trn3='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/invasive/*/'
trn31='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Zebra Ped 1a To Baylor/Sorted Images/Preserved Zebra Ped 1a/*/'
trn11='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Zebra Ped 1a To Baylor/Sorted Images/Not/*/'
trn12='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/To Baylor 2023-01-30/Not Veligers/O1/*/'
trn13='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/To Baylor 2023-01-30/Not Veligers/Z-P/*/'
trn32='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/To Baylor 2023-01-30/Zebra Pediveliger Image1a/Zebra Pediveligers/*/'
trn14='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/USGS Labled Zebra-selected/Preserved Zebra D-Hinge 1 Baylor/Sorted Images/Not/*/'
trn21='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/USGS Labled Zebra-selected/Preserved Zebra D-Hinge 1 Baylor/Sorted Images/Ostracod/*/'
trn33='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/USGS Labled Zebra-selected/Preserved Zebra D-Hinge 1 Baylor/Sorted Images/Zebra D-Hinge/*/'

# trn15='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Baylor 2022-03-21/Davis Dam 2019-07-24/Manually Reviewed/NonVeligers/*/'
# trn34='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Baylor 2022-03-21/Davis Dam 2019-07-24/Manually Reviewed/Veligers/*/'
trn16='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Baylor Lake Minnetonka Chk4 Image1/Not-Invasive Images/*/'
trn35='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Davis Dam 2022-7-25/Imaging Set 1/Labeled Data/Veligers/*/'
trn17='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Davis Dam 2022-7-25/Imaging Set 1/Labeled Data/Not-Veligers/*/'
trn36='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Davis Dam 2022-7-25/Imaging Set 2/Label Data/Veligers/*/'
trn18='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Davis Dam 2022-7-25/Imaging Set 2/Label Data/Not-Veligers/*/'
trn22='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracod Day 2 Image12 To Baylor/Sorted Images/Ostracods/*/'
trn19='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracod Day 2 Image12 To Baylor/Sorted Images/Not/*/'
trn23='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image1 To Baylor/Sorted Images/Ostracods/*/'
trn110='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image1 To Baylor/Sorted Images/Not/*/'
trn24='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image2 To Baylor/Sorted Images/Ostracods/*/'
trn111='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image2 To Baylor/Sorted Images/Not/*/'
trn25='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image3 To Baylor/Sorted Images/Ostracods/*/'
trn112='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image3 To Baylor/Sorted Images/Not/*/'
trn26='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image12 To Baylor/Sorted Images/Ostracods/*/'
trn113='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Veliger Data-selected/Ostracods Day 2 Image12 To Baylor/Sorted Images/Not/*/'
trn37='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Zebra Ped 1 To Baylor/Sorted Images/Pedi-Zebra Veligers/*/'
trn114='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Zebra Ped 1 To Baylor/Sorted Images/Not/*/'
trn27='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Ostracods 1a To Baylor/Sorted Images/Preserved Ostracods 1a/*/'
trn114='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Ostracods 1a To Baylor/Sorted Images/Not/*/'
trn28='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Ostracods 1 To Baylor/Sorted Images/Preserve Ostracods/*/'
trn115='/Users/sadianasrintisha/Desktop/Masters/Code/Dataset/Ostracod vs Pedi-Veliger Examples-selected/Preserved Ostracods 1 To Baylor/Sorted Images/Not/*/'


In [5]:
trn1= glob(trn1)
trn2= glob(trn2)
trn3= glob(trn3)
trn31= glob(trn31)
trn32= glob(trn32)
trn33= glob(trn33)
# trn34= glob(trn34)
trn35= glob(trn35)
trn36= glob(trn36)
trn37= glob(trn37)
trn11= glob(trn11)
trn12= glob(trn12)
trn13= glob(trn13)
trn14= glob(trn14)
# trn15= glob(trn15)
trn16= glob(trn16)
trn17= glob(trn17)
trn18= glob(trn18)
trn19= glob(trn19)
trn110= glob(trn110)
trn111= glob(trn111)
trn112= glob(trn112)
trn113= glob(trn113)
trn114= glob(trn114)
trn115= glob(trn115)
trn21= glob(trn21)
trn22= glob(trn22)
trn23= glob(trn23)
trn24= glob(trn24)
trn25= glob(trn25)
trn26= glob(trn26)
trn27= glob(trn27)
trn28= glob(trn28)

In [6]:
trn1.extend(trn11)
trn1.extend(trn12)
trn1.extend(trn13)
trn1.extend(trn14)
# trn1.extend(trn15)
trn1.extend(trn16)
trn1.extend(trn17)
trn1.extend(trn18)
trn1.extend(trn19)
trn1.extend(trn110)
trn1.extend(trn111)
trn1.extend(trn112)
trn1.extend(trn113)
trn1.extend(trn114)
trn1.extend(trn115)

In [7]:
trn2.extend(trn21)
trn2.extend(trn22)
trn2.extend(trn23)
trn2.extend(trn24)
trn2.extend(trn25)
trn2.extend(trn26)
trn2.extend(trn27)
trn2.extend(trn28)

In [8]:
trn3.extend(trn31)
trn3.extend(trn32)
trn3.extend(trn33)
# trn3.extend(trn34)
trn3.extend(trn35)
trn3.extend(trn36)
trn3.extend(trn37)

In [9]:
tr1= shuffle(trn1)
tr2= shuffle(trn2)
tr3= shuffle(trn3)

tran_index_noninv = np.round( len(tr1)* .6  )
tran_index_osc = np.round( len(tr2)* .6  )
tran_index_inv = np.round( len(tr3)* .6  )


In [10]:
tran_index_noninv
tran_index_inv
tran_index_osc

62.0

In [11]:
print ( "Number of Objects for Non-Invasive, Invasive and Ostracod :" , len(tr1), len(tr3), len(tr2))

Number of Objects for Non-Invasive, Invasive and Ostracod : 5685 1220 104


# Total 7009

# Traindata read

In [12]:
data = []
label = []
breath = []
total = 0

for j in tr3[:(int) (tran_index_inv)]:
    label.append(1)
    a = glob(j+'/*')
    breath.append(len(a))
    total = total + len(a)
    
for j in tr1[:(int) (tran_index_noninv)]:
    label.append(0)
    a = glob(j+'/*')
    breath.append(len(a)) 
    total = total + len(a)
    
for j in tr2[:(int) (tran_index_osc)]:
    label.append(2)
    a = glob(j+'/*')
    breath.append(len(a)) 
    total = total + len(a)

for j in range(0,len(tr3[:(int) (tran_index_inv)])):
    a = glob(tr3[j]+'/*')
    for k in range(0,5):
        data.append(a[k])

for j in range(0,len(tr1[:(int) (tran_index_noninv)])):
    a = glob(tr1[j]+'/*')
    for k in range(0,5):
        data.append(a[k])        

for j in range(0,len(tr2[:(int) (tran_index_osc)])):
    a = glob(tr2[j]+'/*')
    for k in range(0,5):
        data.append(a[k]) 


    

In [55]:
print(label[2])

1


In [13]:
print(total)

135698


In [14]:
imgdata=[]
for i in range(len(data)):
    a = Image.open(data[i])
    b = tf.image.resize_with_crop_or_pad(tf.keras.preprocessing.image.img_to_array(a), 32, 32)
    c = np.array(b)
    imgdata.append(c.reshape(32,32,3))

In [15]:
from tensorflow.keras.utils import to_categorical
idata = np.array(imgdata)
X_train = idata
X_train.shape

(21025, 32, 32, 3)

In [16]:
X_train = X_train.astype('float32') / 255.
X_train.shape


(21025, 32, 32, 3)

In [17]:
X_train = np.reshape(X_train, (len(X_train),32,32,3))
X_train.shape

(21025, 32, 32, 3)

In [18]:
X_train.shape

(21025, 32, 32, 3)

In [19]:
# end= 0
# train_df= []
# breath = 5

# i = 0
# for i in range(0, len(label)):
#     deff = []
#     for k in range(0, (breath)):
        
#         index = (i+k)
        
#         deff.append(X_train[index])
        
#     train_df.append(deff)

# Y_train = to_categorical(label)
# train_df = np.array(train_df)
# YY_Train = label
# np.shape(train_df)

In [20]:
# end= 0
# train_df= []
# breath = 5

# i = 0
# for i in range(0, len(label)):
#     deff = []
#     for k in range(0, (breath)):
        
#         index = (i*5+k)
        
#         deff.append(X_train[index])
        
#     train_df.append(deff)

# Y_train = to_categorical(label)
# train_df = np.array(train_df)
# YY_Train = label
# np.shape(train_df)

In [21]:
Y_train = to_categorical(label)
YY_Train = label

# Train Data 4205


# Test data read

In [22]:
data = []
label = []
breath = []
total = 0

for j in tr3[(int) (tran_index_inv) + 1 :]:
    label.append(1)
    a = glob(j+'/*')
    breath.append(len(a))
    total = total + len(a)
    
for j in tr1[ (int)(tran_index_noninv) + 1:]:
    label.append(0)
    a = glob(j+'/*')
    breath.append(len(a)) 
    total = total + len(a)

for j in tr2[ (int)(tran_index_osc) + 1:]:
    label.append(2)
    a = glob(j+'/*')
    breath.append(len(a)) 
    total = total + len(a)
    
for j in range(0,len(tr3[(int) (tran_index_inv) + 1 :])):
    a = glob(tr3[j]+'/*')
    for k in range(0,5):
        data.append(a[k])

for j in range(0,len(tr1[ (int)(tran_index_noninv) + 1:])):
    a = glob(tr1[j]+'/*')
    for k in range(0,5):
        data.append(a[k])  
        
for j in range(0,len(tr2[ (int)(tran_index_osc) + 1:])):
    a = glob(tr2[j]+'/*')
    for k in range(0,5):
        data.append(a[k])  

In [23]:
imgdata=[]
for i in range(len(data)):
    a = Image.open(data[i])
    b = tf.image.resize_with_crop_or_pad(tf.keras.preprocessing.image.img_to_array(a), 32, 32)
    c = np.array(b)
    imgdata.append(c.reshape(32,32,3))

In [24]:
from tensorflow.keras.utils import to_categorical
idata = np.array(imgdata)
X_test = idata
X_test = X_test.astype('float32') / 255.
X_test = np.reshape(X_test, (len(X_test),32,32,3))

In [25]:
X_test.shape

(14005, 32, 32, 3)

In [26]:
# end= 0
# test_df= []
# breath = 5

# i = 0
# for i in range(0, len(label)):
#     deff = []
#     for k in range(0, (breath)):
        
#         index = (i*5 + k)
        
#         deff.append(X_test[index])
        
#     test_df.append(deff)
    
# Y_test = to_categorical(label)
# test_df = np.array(test_df)
# YY_Test = label
# np.shape(test_df)

In [28]:
Y_test = to_categorical(label)
YY_Test = label


# Test data 2801

In [29]:
random_indices = np.random.choice(X_train.shape[0], Y_train.shape[0], replace=False)

# Select the subset of X_train based on the random indices
X_train_resized = X_train[random_indices]

# Check the new shape
print(X_train_resized.shape)  # (2801, 32, 32, 3)

(4205, 32, 32, 3)


In [30]:
random_indices = np.random.choice(X_test.shape[0], Y_test.shape[0], replace=False)

# Select the subset of X_train based on the random indices
X_test_resized = X_test[random_indices]

# Check the new shape
print(X_test_resized.shape)  # (2801, 32, 32, 3)

(2801, 32, 32, 3)


In [31]:
# Select the first column of Y_train and reshape it
Y_train_resized = np.reshape(Y_train[:, 0], (-1, 1))

# Select the first column of Y_test and reshape it
Y_test_resized = np.reshape(Y_test[:, 0], (-1, 1))

# Check the new shapes
print(Y_train_resized.shape)
print(Y_test_resized.shape)

(4205, 1)
(2801, 1)


In [32]:

print("X_Train data shape:", X_train_resized.shape)
print("X_Test data shape:", X_test_resized.shape)
print("Y_Train data shape:", Y_train_resized.shape)
print("Y_Test data shape:", Y_test_resized.shape)

X_Train data shape: (4205, 32, 32, 3)
X_Test data shape: (2801, 32, 32, 3)
Y_Train data shape: (4205, 1)
Y_Test data shape: (2801, 1)


In [33]:
x_train= X_train_resized
x_test=X_test_resized
y_train=Y_train_resized
y_test=Y_test_resized

In [34]:
print("X_Train data shape:", x_train.shape)
print("X_Test data shape:", x_test.shape)
print("Y_Train data shape:", y_train.shape)
print("Y_Test data shape:", y_test.shape)

X_Train data shape: (4205, 32, 32, 3)
X_Test data shape: (2801, 32, 32, 3)
Y_Train data shape: (4205, 1)
Y_Test data shape: (2801, 1)


In [35]:
num_classes = 3
input_shape = (32, 32, 3)

In [36]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

2023-07-04 11:13:02.812219: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [37]:
def create_encoder():
    input_shape = (32, 32, 3)
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = tf.keras.backend.shape(inputs)
    augmented = tf.reshape(inputs, (augmented[0],) + input_shape)
    augmented = keras.layers.experimental.preprocessing.RandomFlip()(augmented)
    outputs = resnet(augmented)

    model = keras.Model(inputs=inputs, outputs=outputs, name="invasive_noninvasive_ostracod")
    return model

encoder = create_encoder()
encoder.summary()



learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05

Model: "invasive_noninvasive_ostracod"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 tf.compat.v1.shape (TFOpLambda  (4,)                0           ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  ()                  0           ['tf.compat.v1.shape[0][0]']     
 ingOpLambda)                                                                                     
                                                                      

In [38]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="invasive")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [39]:
encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Model: "invasive"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 invasive_noninvasive_ostrac  (None, 2048)             23564800  
 od (Functional)                                                 
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                          

In [41]:
# Evaluate the classifier
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

88/88 [==============================] - 3s 30ms/step - loss: 1.6342 - sparse_categorical_accuracy: 0.7387
Test accuracy: 73.87%


In [42]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="with_projection-head"
    )
    return model

In [43]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()




Model: "with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 invasive_noninvasive_ostrac  (None, 2048)             23564800  
 od (Functional)                                                 
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
Total params: 23,827,072
Trainable params: 23,781,632
Non-trainable params: 45,440
_________________________________________________________________


In [44]:
history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)

Epoch 1/50
16/16 [==============================] - 33s 2s/step - loss: 5.9200
Epoch 2/50
16/16 [==============================] - 30s 2s/step - loss: 5.5721
Epoch 3/50
16/16 [==============================] - 30s 2s/step - loss: 5.5718
Epoch 4/50
16/16 [==============================] - 30s 2s/step - loss: 5.5717
Epoch 5/50
16/16 [==============================] - 30s 2s/step - loss: 5.5714
Epoch 6/50
16/16 [==============================] - 30s 2s/step - loss: 5.5702
Epoch 7/50
16/16 [==============================] - 30s 2s/step - loss: 5.5694
Epoch 8/50
16/16 [==============================] - 31s 2s/step - loss: 5.5709
Epoch 9/50
16/16 [==============================] - 36s 2s/step - loss: 5.5650
Epoch 10/50
16/16 [==============================] - 37s 2s/step - loss: 5.5582
Epoch 11/50
16/16 [==============================] - 35s 2s/step - loss: 5.5510
Epoch 12/50
16/16 [==============================] - 33s 2s/step - loss: 5.5676
Epoch 13/50
16/16 [==============================

In [45]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Epoch 1/50
16/16 [==============================] - 4s 147ms/step - loss: 3.8520 - sparse_categorical_accuracy: 0.7172
Epoch 2/50
16/16 [==============================] - 2s 144ms/step - loss: 2.0279 - sparse_categorical_accuracy: 0.7529
Epoch 3/50
16/16 [==============================] - 2s 144ms/step - loss: 1.3209 - sparse_categorical_accuracy: 0.7555
Epoch 4/50
16/16 [==============================] - 2s 151ms/step - loss: 0.8676 - sparse_categorical_accuracy: 0.7769
Epoch 5/50
16/16 [==============================] - 3s 164ms/step - loss: 0.6168 - sparse_categorical_accuracy: 0.7850
Epoch 6/50
16/16 [==============================] - 2s 145ms/step - loss: 0.5563 - sparse_categorical_accuracy: 0.7990
Epoch 7/50
16/16 [==============================] - 2s 151ms/step - loss: 0.5283 - sparse_categorical_accuracy: 0.8059
Epoch 8/50
16/16 [==============================] - 2s 143ms/step - loss: 0.5277 - sparse_categorical_accuracy: 0.8081
Epoch 9/50
16/16 [==============================

In [78]:
classifier.predict(x_test)

88/88 [==============================] - 3s 30ms/step


array([[2.3178628e-01, 7.6821369e-01, 2.4778897e-08],
       [1.8932779e-01, 8.1066906e-01, 3.2133444e-06],
       [1.8627964e-01, 8.1371748e-01, 2.8406157e-06],
       ...,
       [1.6950706e-01, 8.3049202e-01, 9.1765725e-07],
       [2.3033144e-01, 7.6966470e-01, 3.8517410e-06],
       [1.8856429e-01, 8.1143194e-01, 3.7396089e-06]], dtype=float32)

In [79]:
p = np.round(pred)
p = np.argmax ( p , axis =1)

In [83]:
from sklearn.metrics import accuracy_score


# Calculate accuracy
accuracy = accuracy_score(y_test, p)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 81.15%
